# Nutritional Dataset
Data Exploration of the file

In [27]:
import pandas as pd

# Read CSV file with low_memory=False
nutr_df = pd.read_csv("data.csv", low_memory=False)

In [28]:
nutr_df.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Unnamed: 60
count,3.548200e+04,3.532500e+04,3.588900e+04,3.545200e+04,3.548300e+04,3.560300e+04,3.553800e+04,3.602200e+04,3.557700e+04,3.563000e+04,...,5.425500e+04,5.387800e+04,5.443700e+04,5.709900e+04,5.492200e+04,5.492700e+04,5.292400e+04,5.219900e+04,4.306800e+04,0.0
mean,4.398856e+06,4.477329e+06,4.483279e+06,4.634875e+06,4.727806e+06,4.810557e+06,4.923225e+06,4.958680e+06,5.126162e+06,5.229267e+06,...,1.066701e+08,1.178440e+08,1.194416e+08,1.202183e+08,1.354606e+08,1.381746e+08,1.470674e+08,1.534580e+08,9.105685e+06,NaN
std,4.948245e+07,5.013205e+07,5.054473e+07,5.192122e+07,5.297849e+07,5.401988e+07,5.526006e+07,5.607026e+07,5.763263e+07,5.887277e+07,...,1.017662e+10,1.077350e+10,1.114136e+10,1.132855e+10,1.212198e+10,1.255757e+10,1.312843e+10,1.388442e+10,1.112609e+08,NaN
min,-2.374024e+00,-2.498824e+00,-3.826648e+06,-3.206682e+00,-3.078607e+00,-2.978527e+00,-3.072258e+00,-4.094255e+06,-4.263603e+00,-4.645452e+00,...,-2.272986e+07,-7.720454e+00,-7.829848e+00,-9.816607e+00,-8.166639e+00,-1.578482e+07,-7.726345e+00,-7.709485e+00,-7.784170e+00,NaN
25%,1.907975e+01,2.130000e+01,1.790000e+01,2.228200e+01,2.236877e+01,2.013357e+01,2.196439e+01,1.779723e+01,2.226600e+01,2.237234e+01,...,1.360000e+01,1.351170e+01,1.377504e+01,1.326888e+01,1.410000e+01,1.257937e+01,1.393631e+01,1.387011e+01,1.490000e+01,NaN
50%,5.282000e+03,5.794000e+03,5.158000e+03,5.945000e+03,6.065000e+03,5.998000e+03,6.240000e+03,5.767000e+03,6.438000e+03,6.529500e+03,...,9.608101e+01,9.699493e+01,9.670839e+01,9.155791e+01,9.633862e+01,9.610010e+01,9.900000e+01,9.896778e+01,2.206000e+03,NaN
75%,9.993525e+04,1.034000e+05,1.025840e+05,1.078488e+05,1.107500e+05,1.125765e+05,1.156312e+05,1.151888e+05,1.211970e+05,1.243615e+05,...,8.397350e+04,8.621775e+04,8.342100e+04,6.758400e+04,8.207750e+04,8.399800e+04,9.839350e+04,1.037375e+05,1.802958e+05,NaN
max,3.035056e+09,3.076121e+09,3.129064e+09,3.193947e+09,3.259355e+09,3.326054e+09,3.395866e+09,3.465297e+09,3.535512e+09,3.609910e+09,...,2.253595e+12,2.358301e+12,2.451062e+12,2.546451e+12,2.646913e+12,2.750543e+12,2.833419e+12,2.985748e+12,7.346633e+09,NaN


In [29]:
nutr_df.dtypes

Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
1960              float64
                   ...   
2012              float64
2013              float64
2014              float64
2015              float64
Unnamed: 60       float64
Length: 61, dtype: object

In [30]:
nutr_df.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Unnamed: 60
0,Arab World,ARB,% of females ages 15-49 having comprehensive c...,SH.HIV.KNOW.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,% of males ages 15-49 having comprehensive cor...,SH.HIV.KNOW.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,133.555013,134.159119,134.857912,134.504576,134.105211,133.569626,...,49.999851,49.887046,49.781207,49.672975,49.536047,49.383745,48.796558,48.196418,NaN,NaN
3,Arab World,ARB,Adults (ages 15+) and children (0-14 years) li...,SH.HIV.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,Adults (ages 15+) and children (ages 0-14) new...,SH.HIV.INCD.TL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
print(nutr_df.columns)

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', 'Unnamed: 60'],
      dtype='object')


In [32]:
num_nulls = nutr_df.isna().any(axis=1).sum()
num_nulls

89010

In [33]:
num_dups = nutr_df.duplicated().sum()
num_dups

0

In [34]:
nutr_df.columns
nutr_df.drop("Unnamed: 60", axis=1, inplace=True)

In [35]:
melt_nutr_df = nutr_df.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"], var_name="Year", value_name="Value")


In [39]:
pivoted_df = melt_nutr_df.pivot_table(index='Country Code', columns='Indicator Code', values='Value')
pivoted_df.head()


Indicator Code,NY.GNP.PCAP.CD,SE.ADT.1524.LT.FM.ZS,SE.ADT.1524.LT.MA.ZS,SE.ADT.1524.LT.ZS,SE.ADT.LITR.FE.ZS,SE.ADT.LITR.MA.ZS,SE.ADT.LITR.ZS,SE.ENR.ORPH,SE.PRM.CMPT.FE.ZS,SE.PRM.CMPT.MA.ZS,...,SP.REG.BRTH.UR.ZS,SP.REG.BRTH.ZS,SP.REG.DTHS.ZS,SP.RUR.TOTL,SP.RUR.TOTL.ZG,SP.RUR.TOTL.ZS,SP.URB.GROW,SP.URB.TOTL,SP.URB.TOTL.IN.ZS,SP.UWT.TFRT
Country Code,,,,,,,,,,,,,,,,,,,,,
ABW,NaN,1.001873,99.100614,99.192342,97.101700,97.337924,97.212697,NaN,96.174144,93.762967,...,NaN,NaN,NaN,3.878868e+04,1.525559,51.609375,0.858024,3.535888e+04,48.390625,NaN
ADO,20470.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,80.000,5.268107e+03,1.174219,13.288179,3.820996,4.428900e+04,86.711821,NaN
AFG,480.833333,0.475479,59.031869,45.070434,15.491124,42.399176,29.355613,NaN,9.290589,33.189466,...,35.75,21.7,NaN,1.323962e+07,1.920821,82.559214,4.492917,3.190453e+06,17.440786,NaN
AGO,1610.689655,0.819370,80.460098,72.691472,58.393260,82.346644,69.853556,0.875,36.940429,51.213773,...,39.90,35.6,NaN,8.344457e+06,1.966903,74.761214,5.441774,3.564976e+06,25.238786,NaN
ALB,2037.333333,1.001194,98.998227,99.063629,96.306265,98.244443,97.259558,NaN,96.306297,97.041881,...,98.00,98.3,67.175,1.633051e+06,0.211786,61.328839,2.259171,1.064122e+06,38.671161,7.1


In [13]:
melt_nutr_df.dtypes

Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
Year               object
Value             float64
dtype: object

In [14]:
num_null = melt_nutr_df["Value"].isnull().sum()
print("Number of rows where Value is null:", num_null)

Number of rows where Value is null: 2508518


In [15]:
melt_nutr_df.dropna(subset=['Value'], inplace=True)

In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [17]:
# Extract the features we want to include in the PCA
X = melt_nutr_df[['Year', 'Value']]

In [18]:
# Standardize the features by subtracting the mean and scaling to unit variance
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

# Create a PCA object with 2 components
pca = PCA(n_components=2)

# Fit the PCA model to the standardized data
pca.fit(X_std)

# Transform the data to the new coordinate system
X_pca = pca.transform(X_std)

# Create a new DataFrame with the transformed data and the country labels
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
df_pca['Country'] = melt_nutr_df['Country Name']

# Print the explained variance of each principal component
print(pca.explained_variance_ratio_)

[0.50326287 0.49673713]


In [24]:
import seaborn as sns
import matplotlib.pyplot as plt

In [32]:
# Aggregate the values for duplicate entries
melt_nutr_df_avg = melt_nutr_df.groupby(['Indicator Name', 'Country Name'])['Value'].mean().reset_index()
melt_nutr_df_avg

,Indicator Name,Country Name,Value
0,% of females ages 15-49 having comprehensive c...,Afghanistan,1.500000
1,% of females ages 15-49 having comprehensive c...,Albania,11.333333
2,% of females ages 15-49 having comprehensive c...,Algeria,10.700000
3,% of females ages 15-49 having comprehensive c...,Argentina,45.300000
4,% of females ages 15-49 having comprehensive c...,Armenia,19.033333
...,...,...,...
67995,Women's share of population ages 15+ living wi...,Vietnam,19.073451
67996,Women's share of population ages 15+ living wi...,World,49.354686
67997,Women's share of population ages 15+ living wi...,"Yemen, Rep.",32.095671
67998,Women's share of population ages 15+ living wi...,Zambia,56.782600
